# Activity 1: Whole-Exome Sequencing (WES) Analysis

In this activity, we will analyze whole-exome sequencing data from the [SK-BR-3 breast cancer cell line](https://www.cellosaurus.org/CVCL_0033) to discover cancer mutations. This data was generated using paired-end Illumina sequencing.

We will perform the following steps:

1. Quality control of the sequencing reads
2. Genome alignment with `bwa mem`
3. Calling small somatic variants using `Mutect2`
4. Functional annotation of the variants using `Funcotator`

## Before you start

Make sure...

1. You are in jupyter notebook mode, not jupyter lab 
2. You have the bootcamp kernel active. If not, do `Kernel` > `Change Kernel` > `bootcamp` 
3. You run the cells below to navigate to the correct directory and prevent output wrapping so we can easily visualize the outputs of our commands.

In [ ]:
%% bash

cd ~/bootcamp-02-sequencing/01-whole-exome

In [ ]:
%%html

<style>
div.output_area pre {
    white-space: pre;
}
</style>

## 1. Quality control of the sequencing reads

You can find the FASTQ files containing our WES reads in the directory shown below

```bash
data/
├── wes_illumina_R1.fastq.gz # read 1
└── wes_illumina_R2.fastq.gz # read 2
```

### FASTQ format

Let's take a look at the first few lines of each file. Since the files are compressed, we will first decompress using `zcat` and then pipe the output to `head` to display the first 10 lines. 

Run the following commands, compare the output with the image below, and consider the following questions.

1. Do you see the canonical four lines of a fastq file? 
2. What do you notice about the read IDs in corresponding lines of each file?

![](./img/fastq_format.jpg)

You can find a complete description of the FASTQ format specification here: [Cock et al. "The Sanger FASTQ file format for sequences with quality scores, and the Solexa/Illumina FASTQ variants 
" *Nucleic Acids Research* 2010](https://academic.oup.com/nar/article/38/6/1767/3112533).

In [ ]:
%%bash

# read 1 file
zcat data/wes_illumina_R1.fastq.gz | head

In [ ]:
%%bash

# read 2 file
zcat data/wes_illumina_R2.fastq.gz | head

### WES FASTQ quality check

Now let's use [`fastqc`](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/) to perform a quality check of the raw sequencing data. FASTQC will generate a report in HTML format, which you can open in your browser. To save time, we have already run `fastqc` and saved the reports. In your jupyter browser, you can find the resultant HTML files at `~/bootcamp-02-sequencing/01-whole-exome/data/wes_illumina_R*.fastqc.html`. You can right-click to open them and view in your browser.

Take a look and the report and answer the following questions:


1. Are there any warnings or failures in the report? If so, inspect them.
2. What is the sequence length distribution of reads? Are they the same for both files?
3. What is the per base sequence distribution? What is the relationship between quality score and position in read? Why do you think this is?

You can also compare your report to examples for [Good Illumina data](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/good_sequence_short_fastqc.html) and [Bad Illumina data](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/bad_sequence_fastqc.html).

## 2. Genome alignment with `bwa mem`

To align the reads to the reference genome, we will use the [Burrows-Wheeler Aligner (BWA) Maximal Exact Match (MEM)](http://bio-bwa.sourceforge.net/) algorithm. BWA-MEM is a fast and accurate aligner for short reads and is the current gold standard for Illumina short-read genome alignment, although it will likely soon be replaced by Illumina's DRAGMAP method (you can read more about DRAGMAP vs BWA-MEM benchmarks [here](https://gatk.broadinstitute.org/hc/en-us/articles/4410953761563-Introducing-DRAGMAP-the-new-genome-mapper-in-DRAGEN-GATK)). Sequence and genome alignment is a heavily studied computational problem and will be covered in Pavel Pevzer's course "Bioinformatics II (BENG 202/CSE 282). Introduction to Bioinformatics Algorithms".


<!-- TODO: add command for bwa mem, but dont have them run it -->
<!-- TODO: add a note about reference genomes. have them peek at the fasta file -->

To save time, we have already run `bwa mem` and saved the alignments in the binary alignment/map (BAM) format, a compressed version of the sequence alignment/map (SAM) format. You can find the resultant BAM file at `data/wes_illumina.bam`.

### Sequence Aligment/Map (SAM) format

Let's take a look at the BAM file header and the first 10 alignments. We will use [`samtools`](http://www.htslib.org/doc/samtools.html) to do automatically decompress and display the file contents. 

Run the following commands, check the outputs with the image below, and considering the following questions

1. Do you see the canonical features of a SAM file? 
2. What is stored in the header? 
3. Do you noticed any FASTQ file fields that are retained in the SAM format?

![](./img/sam_format.jpg)

You can find a complete description of the SAM format specification here: [samtools.github.io/hts-specs/SAMv1.pdf](https://samtools.github.io/hts-specs/SAMv1.pdf).

In [ ]:
%%bash

# check the file header
samtools view -H data/wes_illumina.bam 

In [ ]:
%%bash

# check the first 10 alignments
samtools view data/wes_illumina.bam | head

### WES alignment statistics

Now, to understand the quality of the sequencing experiment and alignment, let's extract some summary statistics from the BAM file. `samtools` is a great tool for this. Let's first see how many reads mapped to each contig (contiguous sequence) of the reference genome using the `idxstats` subcommand. 

Here is description of the output from the [`samtools idxstats` manual page](http://www.htslib.org/doc/samtools-idxstats.html)

>The output is TAB-delimited with each line consisting of reference sequence name, sequence length, # mapped read-segments and # unmapped read-segments. It is written to stdout. Note this may count reads multiple times if they are mapped more than once or in multiple fragments.

Considering the following questions:

1. Which contigs have the most/least mapped read-segments?
2. Which contigs ahve the most/least unmapped read read-segments?
3. Why are there more than 24 contigs?

In [ ]:
%%bash

# number of reads mapped to each contig
# pipe to column -t to format the output for easy reading
samtools idxstats data/wes_illumina.bam | column -t

The FLAG column corresponds to bitwise flags that describe up to 13 features about each alignment, including if the read is paired, if the read and its mate are mapped or unmapped, the orientation of the read and its mate relative to the genome, and more. These flags are commonly used to evaluate the quality of the alignment. The Broad Institute has a simple online tool to decode SAM flags, available here [broadinstitute.github.io/picard/explain-flags.html](https://broadinstitute.github.io/picard/explain-flags.html).

We can use `samtools` again, this time using the `flagstat` subcommand to look at the number of reads that have each flag. Since this flagstat requires a full pass through the file, it can take a few minutes to run. You can read about this command on the [`samtools flagstat` manual page](http://www.htslib.org/doc/samtools-flagstat.html)

Run the command below and consider the following questions:

1. What is the difference between a primary and secondary/supplentary alignment?
2. What are duplicates?
3. Why might a read's mate be mapped to a different chromosome?

In [ ]:
%%bash

# number of reads categorized by each type of SAM flag
# let's specify using 8 threads to speed up the computation.
# this should take about 5 minutes to run
samtools flagstat -@ 8 data/wes_illumina.bam

### WES aligment visualization

Now let's visualize the alignment along a known oncogene *ERBB2* (aka HER2) using the interactive genomics viewer (IGV). Run the code cell below. After the viewer loads, zoom in to view the reads and coverage. 

What do you notice about the distribution of reads along this gene? You can read about how to interpret the alignment visualization on the IGV website's ["Viewing Alignments" page](https://software.broadinstitute.org/software/igv/AlignmentData).

In [ ]:
# Must be in Jupyter notebook (not lab) for this to work
import igv_notebook
import os
igv_notebook.init() 

browser = igv_notebook.Browser(
    {
        "genome": "hg19",
        "locus": "chr17:7,569,893-7,579,467" # TP53 gene
    }
)

browser.load_track(
    {
        "name": "Alignment",
        "path": "data/wes_illumina.bam",
        "indexPath": "data/wes_illumina.bam.bai",
        "format": "bam",
        "type": "alignment"
    })

## 3. Calling small somatic variants using `Mutect2`

To determine which genes are mutated in this cell line, we will perform **variant calling**. By examining the differences between the reference genome and the aligned reads, we can identify regions of the genome that are different between the reference and the sample and call these regions as variants. [Mutect2](https://gatk.broadinstitute.org/hc/en-us/articles/360037225632-Mutect2) is a somatic short variant caller that is part of the [Genome Analysis Toolkit (GATK)](https://gatk.broadinstitute.org/hc/en-us), designed to call short nucleotide variants (SNVs) and insertions/deletions (indels). Like sequencing alignment, variant calling is a heavily studied computational problem and is covered in Melissa Gymrek's course "CSE 284. Personal Genomics for Bioinformaticians".

To save time, we have already run `Mutect2` on our whole-exome sequencing BAM file. You can find the resultant compressed VCF file at `data/wes_illumina_mutect.vcf.gz`.

### Variant Call Format (VCF)

Let's look at the top of the VCF file. We will use [`bcftools`](http://samtools.github.io/bcftools/bcftools.html) to automatically decompress and display the VCF. Run the following commands, check the output with the image below, and consider the following questions.

1. Do you see the canonical features of a VCF file?
2. What is stored in the header? How is it different than the SAM header?
3. Do you noticed any FASTQ file fields that are retained in the VCF?

![](./img/vcf_format.jpg)

You can find a complete description of the VCF specification here: [samtools.github.io/hts-specs/VCFv4.2.pdf](https://samtools.github.io/hts-specs/VCFv4.2.pdf).

In [ ]:
%%bash

# check the file header
bcftools view -h data/wes_illumina_mutect.vcf.gz

In [ ]:
%%bash 

# check the first 10 variants
bcftools view -H data/wes_illumina_mutect.vcf.gz | head

### WES variant calling statistics

Now let's look at the variant statistics using [`bcftools stats`](http://samtools.github.io/bcftools/bcftools.html#stats).

In [ ]:
%%bash

bcftools stats data/wes_illumina_mutect.vcf.gz 

Let's now load the variants into IGV to visualize them. Run the code cell below. After the viewer loads, zoom in to view the variants along *ERBB2*. 

What do you notice about the distribution of variants along this gene? You can read about how to interpret the variant visualization on the IGV website's ["VCF Files" page](https://software.broadinstitute.org/software/igv/viewing_vcf_files).

In [ ]:
import igv_notebook
import os
igv_notebook.init() 

browser = igv_notebook.Browser(
    {
        "genome": "hg19",
        "locus": "chr17:7,569,893-7,579,467" # TP53 gene
    }
)

browser.load_track(
    {
        "name": "Alignment",
        "path": "data/wes_illumina.bam",
        "indexPath": "data/wes_illumina.bam.bai",
        "format": "bam",
        "type": "alignment"
    })

browser.load_track(
    {
        "name": "Variant calls",
        "path": "data/wes_illumina_mutect.vcf.gz",
        "indexPath": "data/wes_illumina_mutect.vcf.gz.tbi",
        "format": "vcf",
        "type": "variant"
    })

## 4. Functional annotation of the variants using `Funcotator`

While `Mutect2` discovers variants, it does not annotate them with information about how the variant may affect a gene's function. To do this, we used the [Funcotator (FUNCtional annOTATOR)](https://gatk.broadinstitute.org/hc/en-us/articles/360035889931-Funcotator-Information-and-Tutorial) tool, which is also part of GATK.

To save time, we have already run `Mutect2` on our whole-exome sequencing BAM file. You can find the resultant compressed VCF file at `data/wes_illumina_mutect.funcotated.vcf.gz`.

Look through the VCF, what has changed?

In [ ]:
%%bash

# check the file header
bcftools view -h data/wes_illumina_mutect.funcotated.vcf.gz

In [ ]:
%%bash 

# check the first 10 variants
bcftools view -H data/wes_illumina_mutect.funcotated.vcf.gz | head

In [ ]:
%%bash

# lets look at TP53
bcftools view --regions "chr17:7569893-7579467" data/wes_illumina_mutect.funcotated.vcf.gz